In [1]:
import numpy as np  # http://www.numpy.org
import matplotlib.pyplot as plt   # http://matplotlib.org
import math as M
from scipy.optimize import fsolve

In [2]:
############ Initilisation #################
############################################

#### Constants #####
L_x = 1e6          # [m] zonal extent 
L_y = 2e6          # [m] meridional extent 
A = L_x*L_y        # [m^2] surface area of interior 
P = 2*L_x+2*L_y    # [m] Perimeter 

g = 9.81           # [m/s^2] gravitational acceleration 
C_p = 3.994e3        # [J/(kg K)] specific heat capacity of sea water 
c = 0.006          # [] bottom slope coefficient
s = 0.02           # [] average bottom slope around the perimeter of the marginal sea

#### Parameters ####
H = 1000           # [m] sill height 
f_0 = 1.2e-4       # [s^-1] Coriolis parameter 
Gamma = 20         # [W/(m^2 C)] Restoring strength 
alpha_T = 0.2      # [kg m^-3 C^-1] thermal expansion coefficient; source: Ypma[2020]
alpha_S = 0.8      # [kg m^-3] Haline expansion coefficient 
#T_1 = 8.87         # [Celsius] ingoing temperature 
T_A = 2.8          # [Celsius] Atmospheric temperature 
ro = 1028          # [kg m^-3] density sea water 
S = 35             # [ppt] net evaporation minus precipitation
#S_1 = 35.2         # [g kg^-1] incoming salinity; source Lambert                          ! is ppt = g kg^-1 ???
E = -2e-8          # [m/s] a representative ocean salinity; source: Spall[2015]

#### functions ####
L = H/s           # [m] width of sloping topography of the boundary current 
#L = 75e3

In [14]:
steps=7          # test values
dx=10            # test values

T_0 = np.zeros(steps)
S_0 = np.zeros(steps)
V = np.zeros(steps)
T_1 = np.zeros(steps)
S_1 = np.zeros(steps)

T_1[0] = 8.87 
S_1[0] = 35.2 

for i in range(steps-1):
    def main(x): # function with seven unkowns:  T_0,  S_0,    V, T_1+1, S_1+1,  V_+1 
                 #                              x[0], x[1], x[2],  x[3],  x[3],  x[5]
        return [c*x[2]*(T_1[i]-x[0])-A*Gamma*(x[0]-T_A)/(ro*C_p),
                c*x[2]*(T_1[i]-x[0])-A*E*S,
                x[3]-x[2]*(T_1[i]-x[0])*c*dx/(H*L)-(2*x[2]*T_1[i]-T_1[i]*x[5])/x[2],
                x[5]-x[2]*(T_1[i]-x[0])*c*dx/(H*L)-(2*x[2]*T_1[i]-x[2]*x[3])/T_1[i],
                x[3]-x[2]*(S_1[i]-x[1])*c*dx/(H*L)-(2*x[2]*S_1[i]-S_1[i]*x[5])/x[2],
                x[5]-x[2]*(S_1[i]-x[1])*c*dx/(H*L)-(2*x[2]*S_1[i]-x[2]*x[3])/S_1[i]]

    [T_0[i],S_0[i],V[i],T_1[i+1],S_1[i+1],V[i+1]] = fsolve(main, [1,1,1,1,1,1]) 
    # V[i] and V[i+1] are calculated at the same time - that's not so good I think
    
T_0,S_0,V,T_1,S_1,V
    

(array([ 2.79999631e+00,  2.80000000e+00,  2.79958905e+00,  2.79944763e+00,
        -5.44872850e+69,  1.00000000e+00,  0.00000000e+00]),
 array([ 1.31728110e+09, -6.68330352e+10,  9.50345523e+17,  2.66228990e+82,
         1.32642675e+83,  1.00000000e+00,  0.00000000e+00]),
 array([-1.25193128e+03,  1.68182762e-01,  2.13041766e+01,  7.47118006e-01,
         1.00000001e+00,  1.00000000e+00,  1.00000000e+00]),
 array([ 8.87000000e+00,  2.70829156e+01, -9.84288225e+01, -8.38254000e+05,
        -2.22441782e+60,  2.44744664e+70,  1.00000000e+00]),
 array([ 3.52000000e+01, -1.28738046e+10,  9.50345331e+17,  2.98827731e+83,
         1.89309800e+85, -6.53421928e+82,  1.00000000e+00]),
 array([-1.25193128e+03,  1.68182762e-01,  2.13041766e+01,  7.47118006e-01,
         1.00000001e+00,  1.00000000e+00,  1.00000000e+00]))

In [17]:
# only V[i+1] being calculated - maybe works better?

steps=7          # test values
dx=10            # test values

T_0 = np.zeros(steps)
S_0 = np.zeros(steps)
V = np.zeros(steps)
T_1 = np.zeros(steps)
S_1 = np.zeros(steps)

T_1[0] = 8.87 
S_1[0] = 35.2 
V[0] = 0.35

for i in range(steps-1):
    def main(x): # function with seven unkowns:  T_0,  S_0,  T_1+1, S_1+1,  V_+1 
                 #                              x[0], x[1],   V[2],  x[3],  x[4],
        return [c*V[i]*(T_1[i]-x[0])-A*Gamma*(x[0]-T_A)/(ro*C_p),
                c*V[i]*(T_1[i]-x[0])-A*E*S,
                x[2]-V[i]*(T_1[i]-x[0])*c*dx/(H*L)-(2*V[i]*T_1[i]-T_1[i]*x[4])/V[i],
                x[4]-V[i]*(T_1[i]-x[0])*c*dx/(H*L)-(2*V[i]*T_1[i]-V[i]*x[2])/T_1[i],
                x[2]-V[i]*(S_1[i]-x[1])*c*dx/(H*L)-(2*V[i]*S_1[i]-S_1[i]*x[4])/V[i]]

    [T_0[i],S_0[i],T_1[i+1],S_1[i+1],V[i+1]] = fsolve(main, [1,1,1,1,1]) 
    # V[i] and V[i+1] are calculated at the same time - that's not so good I think
T_0,S_0,V,T_1,S_1,V
    

(array([2.8       , 2.79999999, 2.8       , 2.79999999, 2.79999998,
        2.79999997, 0.        ]),
 array([-7.98562205e+10, -1.27545199e+12, -6.22256913e+13, -8.24463361e+16,
         6.31536128e+21, -6.51649048e+23,  0.00000000e+00]),
 array([ 0.35      ,  1.14805243,  2.29610483,  4.59220995,  9.18440317,
        18.35584019, 36.71169414]),
 array([ 8.87000000e+00, -1.14871396e+01, -2.28778714e-07,  9.88924333e-15,
         3.59915139e-20,  5.08111801e-23,  7.41504305e-22]),
 array([ 3.52000000e+01, -8.95498653e+10,  1.42968988e+12, -1.24531756e+14,
         4.93022563e+16,  1.97193768e+22, -2.92749229e+22]),
 array([ 0.35      ,  1.14805243,  2.29610483,  4.59220995,  9.18440317,
        18.35584019, 36.71169414]))